<a href="https://colab.research.google.com/github/21cs076/Real-Time-Landslide-Detection-and-Buried-Vehicle-Identification-using-YOLOv8/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics
!pip install roboflow

from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO('yolov10n.pt')

# Perform inference on an image from Google Drive
results = model('https://ultralytics.com/images/bus.jpg')

# Iterate through the results and display each
for result in results:
    result.show()

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="4MptKsSsONT0fnQsIDc2")
project = rf.workspace("deslizamentos").project("landslide-detection-augmentation")
version = project.version(12)
dataset = version.download("yolov8")


In [ ]:
from ultralytics import YOLO

# Load a pre-trained YOLOv8 model
model = YOLO("yolov8n.pt")

# Define early stopping parameters
patience = 10
best_loss = float('inf')
counter = 0

# Custom training loop with early stopping
for epoch in range(135):
    print(f"Epoch {epoch+1}/135")

    # Train the model for one epoch
    model.train(
        data="/content/Landslide-Detection-Augmentation-12/data.yaml",
        epochs=1,
        batch=16,
        imgsz=640,
        project="runs/train",
        name="landslide_detection",
        exist_ok=True
    )

    # Validate the model and get the validation loss
    results = model.val(data="/content/Landslide-Detection-Augmentation-12/data.yaml", task="detect")
    val_loss = results['metrics']['box']['val_loss']

    # Check if the validation loss has improved
    if val_loss < best_loss:
        best_loss = val_loss
        counter = 0
        # Save the model weights if validation loss improves
        model.save("/content/Landslide-Detection-Augmentation-12/best10.pt")
    else:
        counter += 1

    # Early stopping
    if counter >= patience:
        print("Early stopping triggered")
        break

print("Training completed")

In [ ]:
from google.colab import files
uploaded = files.upload()

import os
os.listdir('/content')

In [23]:
from ultralytics import YOLO

# Load the trained YOLO model
model = YOLO("/content/best.pt")
# Evaluate the model using the val() method
metrics = model.val(data="/content/Landslide-Detection-Augmentation-12/data.yaml", task="detect")

# Print evaluation metrics
print(f"mAP: {metrics.box.maps}")
print(f"mAP50: {metrics.box.map50}")
print(f"mAP75: {metrics.box.map75}")  # mAP at IoU thresholds of 50% (mAP50) and 75% (mAP75)

Ultralytics 8.3.56 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv10n summary (fused): 285 layers, 2,695,196 parameters, 0 gradients, 8.2 GFLOPs


val: Scanning /content/Landslide-Detection-Augmentation-12/valid/labels.cache... 115 images, 37 backgrounds, 0 corrupt: 100%|██████████| 115/115 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:24<00:00,  3.05s/it]


                   all        115        162      0.649      0.415      0.445      0.205
               Fissure         29         60      0.558       0.35      0.378      0.179
             Landslide         61        102      0.741       0.48      0.513      0.231
Speed: 2.4ms preprocess, 182.7ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs/detect/val6
mAP: [    0.17895     0.23135]
mAP50: 0.44538361500741763
mAP75: 0.1626047778303505


In [ ]:
!pip install onnx
!pip install onnxruntime-gpu

import torch
import onnxruntime as ort
import numpy as np

# Convert to ONNX format
onnx_model_path = "/content/Landslide-Detection-Augmentation-12/lsmodel.onnx"
torch.onnx.export(
    model.model,
    torch.randn(1, 3, 640, 640),
    onnx_model_path,
    opset_version=11,
    input_names=["input"],
    output_names=["output"]
)

print(f"Model has been converted to ONNX format and saved at: {onnx_model_path}")

# Load and run inference using ONNX model
onnx_session = ort.InferenceSession(onnx_model_path)
outputs = onnx_session.run(None, {'input': np.random.rand(1, 3, 640, 640).astype(np.float32)})

print(outputs)

In [ ]:
# Clone YOLOv8 repository
!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics
